In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

tokenizer = AutoTokenizer.from_pretrained("sshleifer/distilbart-cnn-6-6")

model = AutoModelForSeq2SeqLM.from_pretrained("sshleifer/distilbart-cnn-6-6",torchscript=True)

# model = BartModel.from_pretrained("sshleifer/bart-tiny-random")
input_ids = decoder_input_ids = torch.tensor([19 * [1] + [model.config.eos_token_id]])
traced_model = torch.jit.trace(model, (input_ids, decoder_input_ids))

traced_model.save("distilbart.pt")

/Users/philipp/Projects/huggingface/transformers/src/transformers/models/bart/configuration_bart.py:177: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions.The config can simply be saved and uploaded again to be fixed.
  warnings.warn(


/Users/philipp/Projects/huggingface/transformers/src/transformers/models/bart/modeling_bart.py:213: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert attn_weights.size() == (
/Users/philipp/Projects/huggingface/transformers/src/transformers/models/bart/modeling_bart.py:220: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert attention_mask.size() == (
/Users/philipp/Projects/huggingface/transformers/src/transformers/models/bart/modeling_bart.py:252: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can

In [12]:
ARTICLE_TO_SUMMARIZE = "My friends are cool but they eat too many carbs."

inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


['My friends']


In [17]:
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors='pt')
# Generate Summary
summary_ids = traced_model.generate(inputs['input_ids'], num_beams=4, max_length=5, early_stopping=True)
print([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

RuntimeError: forward() expected at most 3 argument(s) but received 5 argument(s). Declaration: forward(__torch__.transformers.models.bart.modeling_bart.BartForConditionalGeneration self, Tensor input_ids, Tensor mask) -> ((Tensor, ((Tensor, Tensor, Tensor, Tensor), (Tensor, Tensor, Tensor, Tensor), (Tensor, Tensor, Tensor, Tensor), (Tensor, Tensor, Tensor, Tensor), (Tensor, Tensor, Tensor, Tensor), (Tensor, Tensor, Tensor, Tensor)), Tensor))